In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import random
import json
import requests


: 

In [3]:
#先运行命令再运行此代码块 & Start-Process "C:\Program Files\Google\Chrome\Application\chrome.exe" -- " --remote-debugging-port=9530 --user-data-dir=""D:\twitter_data"""
chromedriver_path = r"C:\Users\admin\chromedriver-win64\chromedriver-win64\chromedriver.exe"

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9530")

# 创建 ChromeDriver 服务
service = Service(chromedriver_path)

# 初始化 WebDriver
driver = webdriver.Chrome(service=service, options=option)

# 去除 WebDriver 痕迹
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    });
    """
})

# 设置隐式等待
driver.implicitly_wait(10)

# 测试访问页面
driver.get("https://www.x.com")
print("标题:", driver.title)


标题: (1) Home / X


In [13]:
from datetime import datetime
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
import os
import re
from datetime import datetime
from bs4 import BeautifulSoup
import time




def get_posts_byday(keyword, start_date, end_date, is_live): 
    posts_lists = []
    post_index = set() #帖子独立标识集合去重
    # if is_live:
    #     driver.get(f"https://x.com/search?q={keyword}%20lang%3Aen%20until%3A{end_date}%20since%3A{start_date}&src=typed_query&f=live") # 查询条件 src=typed_query：表示这是一个手动输入的查询。f=live：表示只检索实时搜索结果
    # else:
    #     driver.get(f"https://x.com/search?q={keyword}&src=typed_query&f=top") # 查询条件 src=typed_query：表示这是一个手动输入的查询。
    driver.get(f"https://x.com/Ruslan558775")
    time.sleep(5)  # 等待页面加载完成


    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = 4500  # 每次向下滚动的像素数
    scroll_count = 0
    previous_post_count = 0  # 用于记录上一次的帖子ID数量
    max_attempts = 5  # 当数据条数不变的最大允许次数
    attempts = 0
    garbages = []
    isgarbage = 0

    # 保存路径
    base_dir = r"C:\Users\admin\codespace\twittercrawler\twitter\json"
    os.makedirs(base_dir, exist_ok=True)
    if is_live:
        posts_file = os.path.join(base_dir, f"[latest rank]posts_{keyword}_{start_date}_{end_date}.json")
    else:
        posts_file = os.path.join(base_dir, f"[top rank]posts_{keyword}_{start_date}_{end_date}.json")

        while True:
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'lxml')
            post_tags = soup.select('div[data-testid="cellInnerDiv"]')
            # post_tags = soup.select('article[data-testid="tweet"]')
            for post_tag in post_tags:
                transform_value = post_tag['style'].split('translateY(')[1].split('px)')[0]
                # print(transform_value)
                if float(transform_value) > 450 and transform_value not in post_index: # 不想漏掉第一个帖子，所以宁可错采一些
                    post_index.add(transform_value)

                    # 提取用户名
                    username_tag = post_tag.select_one('div[data-testid="User-Name"]')
                    if username_tag:
                        username = username_tag.get_text(strip=True)
                        # 检查用户名是否包含垃圾词
                        if any(garbage in username for garbage in garbages):
                            isgarbage = 1
                            continue  # 跳过当前帖子，不再提取ID
                        username = username_tag.select_one('span').text.strip() if username_tag.select_one('span') else ''
                    isgarbage = 0

                    post_content = ""
                    tweet_text_div = post_tag.select_one('div[data-testid="tweetText"]')
                    if tweet_text_div:
                        # 遍历所有子元素，按顺序提取文字、图片和链接
                        for element in tweet_text_div.children:
                            if element.name == 'span':  # 处理文本
                                post_content += element.text.strip() + " "
                            elif element.name == 'img':  # 处理图片
                                alt_text = element.get('alt', '')
                                if alt_text:
                                    post_content += f"{alt_text} "
                            elif element.name == 'a':  # 处理链接
                                href = element.get('href', '')
                                if href:
                                    post_content += f"{href} "
                                    post_content += element.text.strip() + " "

                    # 输出最终提取的内容
                    # print(post_content)

                    # print(post_content)

                    # 查找包含aria-label的div标签，提取转评赞以及帖子id
                    if start_date < '2023-01-01':
                        aria_label = post_tag.find('div', attrs={'aria-label': re.compile(r'.*likes.*')}) # likes views
                    else:
                        aria_label = post_tag.find('div', attrs={'aria-label': re.compile(r'.*views.*')}) # likes views
                    if aria_label:
                        # 从aria-label中提取数据
                        label_text = aria_label['aria-label']

                        # 初始化数据
                        replies = reposts = likes = bookmarks = views = 0

                        if 'repl' in label_text:
                            match = re.search(r'(\d+)\s+repl', label_text)
                            replies = int(match.group(1)) if match else 0

                        if 'repost' in label_text:
                            match = re.search(r'(\d+)\s+repost', label_text)
                            reposts = int(match.group(1)) if match else 0

                        if 'like' in label_text:
                            match = re.search(r'(\d+)\s+like', label_text)
                            likes = int(match.group(1)) if match else 0

                        if 'bookmark' in label_text:
                            match = re.search(r'(\d+)\s+bookmark', label_text)
                            bookmarks = int(match.group(1)) if match else 0
                        if 'view' in label_text:
                            # views = int(re.search(r'(\d+)\s+view', label_text).group(1))
                            # 检查是否匹配到了 "view" 和数字
                            match = re.search(r'(\d+)\s+view', label_text)
                            if match:
                                views = int(match.group(1))
                            else:
                                views = 0  # 如果没有匹配到，设置默认值为0

                        # 查找帖子ID和用户ID
                        post_id_tag = aria_label.find_all('div', class_='css-175oi2r r-18u37iz r-1h0z5md r-13awgt0')
                        if post_id_tag:
                            # 找到第四个div，提取其中的链接
                            post_id_link = post_id_tag[3].find('a', href=True)
                            if post_id_link:
                                post_id = post_id_link['href'].split('/')[3]  # 从URL中提取帖子ID
                                user_id = post_id_link['href'].split('/')[1]
                            else:
                                post_id = ''
                                user_id = ''
                        else:
                            post_id = ''
                            user_id = ''
                    else:
                        replies = reposts = likes = bookmarks = views = 0
                        post_id = ''
                        user_id = ''
                        break

                    # 提取帖子发布时间
                    time_tag = post_tag.find('time', attrs={'datetime': True})
                    if time_tag and 'datetime' in time_tag.attrs:
                        datetime_str = time_tag['datetime']
                        # 解析ISO格式的日期时间字符串
                        dt = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ')
                        # 格式化为“年-月-日 时:分:秒”格式
                        post_time = dt.strftime('%Y-%m-%d %H:%M:%S')
                    else:
                        post_time = ''

                    # 提取hashtags，确保唯一性，使用集合去重
                    hashtags = set()
                    hashtag_tags = post_tag.select('a[href*="/hashtag/"]')
                    for tag in hashtag_tags:
                        hashtag = tag.get_text(strip=True)
                        hashtags.add(hashtag)

                    # 提取图片和视频的链接
                    media_urls = set()  # 使用集合来自动去重
                    img_tags = post_tag.find_all('img')
                    for img in img_tags:
                        img_url = img.get('src')
                        if img_url and 'profile_images' not in img_url and 'emoji' not in img_url:
                            media_urls.add(img_url)
                    media_urls = list(media_urls)

                    # 提取@了哪些用户
                    # at_usernames = set()  # 使用集合确保唯一性
                    # at_usernames.update(re.findall(r'@(\w+)', post_content))  # 查找所有 @ 后的用户名

                    # 提取帖子中的所有链接（使用正则表达式从文本中匹配URL）
                    urls = set()  # 使用集合去重

                    # 匹配以 http 或 https 开头的 URL
                    url_pattern = re.compile(r'https?://[^\s]+')

                    # 在帖子文本中查找所有符合条件的URL
                    urls.update(url_pattern.findall(post_content))

                    # 查找回复的多个用户
                    reply_users = []
                    reply_tag = post_tag.find('div', class_='css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41') # 查找包含回复信息的 div
                    if reply_tag:
                        # 获取所有回复用户的 a 标签
                        reply_user_tags = reply_tag.find_all('a')
                        for reply_user_tag in reply_user_tags:
                            # 提取用户名并添加到列表
                            reply_users.append(reply_user_tag.text.strip())
                    # 构建帖子数据，新增字段
                    web_object = {
                        "postUrl": f"https://x.com/{user_id}/status/{post_id}",
                        "text": post_content,
                        "mid": post_id,
                        "userName": username,
                        "userId": user_id,
                        "date": post_time,
                        "likeNum": likes,
                        "commentNum": replies,
                        "repostNum": reposts,
                        "viewNum": views,
                        "bookmarks": bookmarks,
                        "hashtags": list(hashtags),
                        "mediaUrls": media_urls,
                        # "atUsernames": list(at_usernames),  # 新增字段：记录@的用户
                        # "urls": list(urls),  # 新增字段：记录帖中的所有链接
                        # "replyUsers": reply_users  # 新增字段：记录多个回复的用户
                    }

                    #  # 构建帖子数据
                    # web_object = {
                    #     "text": post_content,
                    #     "mid": post_id,
                    #     "userName": username,
                    #     "userId": user_id,
                    #     "date": post_time,
                    #     "likeNum": likes,
                    #     "commentNum": replies,
                    #     "repostNum": reposts,
                    #     "viewNum": views,
                    #     "bookmarks": bookmarks
                    # }

                    posts_lists.append(web_object)

                    # 保存 posts
                    if os.path.exists(posts_file):
                        # File exists, so read the existing data
                        with open(posts_file, "r", encoding="utf-8") as f:
                            existing_data = json.load(f)
                        
                        # Append the new posts data to the existing data
                        existing_data.extend(posts_lists)

                        # Write the updated data back to the file
                        with open(posts_file, "w", encoding="utf-8") as f:
                            json.dump(existing_data, f, ensure_ascii=False, indent=4)
                        
                        print(f"Data appended to {posts_file}")

                    else:
                        # File doesn't exist, create it and write the new data
                        with open(posts_file, "w", encoding="utf-8") as f:
                            json.dump(posts_lists, f, ensure_ascii=False, indent=4)

                        print(f"File {posts_file} created with initial data.")
                    '''
                    [目前]存储在posts.json文件中的每个帖子对象包含以下信息：

                        text: 帖子的正文内容。
                        mid: 帖子的ID。
                        userName: 用户名。
                        userId: 用户ID。
                        date: 帖子的发布时间。
                        likeNum: 点赞数。
                        commentNum: 评论数。
                        repostNum: 转发数。
                        viewNum: 浏览数。
                        bookmarks: 收藏数。
                    ''' 
                
            current_post_count = len(posts_lists)
            print(f"已收集到 {current_post_count} 个帖子")

            # if current_post_count >= 200:  # 如果爬取到的数据条数超过200，则停止爬取
            #     break

            if current_post_count == previous_post_count and not isgarbage: #防止遇到批量垃圾帖子，导致误识别为滑到底了
                time.sleep(random.uniform(1, 2))
                attempts += 1
            else:
                attempts = 0
                previous_post_count = current_post_count

            if attempts == max_attempts:
                # 检测到页面错误信息，尝试点击'Retry'按钮...
                if len(post_tags) > 0 and post_tags[-1].select_one('span').text.strip() == "Something went wrong. Try reloading." if post_tags[-1].select_one('span') else '':
                    print(f"检测到'Retry'按钮，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')},滚动了{scroll_count}次")
                    # 等待人工确认点击
                    input("按Enter键以继续爬取...")
                    while True:  # 无限循环，直到成功点击
                        try:
                            # 等待6分钟（360秒）
                            print("等待1分钟...")
                            time.sleep(60)  # 等待6分钟

                            # 使用 CSS Selector 或 XPath 查找'Retry'按钮并确保它可点击
                            retrybutton = WebDriverWait(driver, 3).until(
                                EC.element_to_be_clickable((By.XPATH, "//button[.//*[contains(text(),'Retry')]]"))
                            )

                            # 滚动到按钮所在位置
                            driver.execute_script("arguments[0].scrollIntoView();", retrybutton)

                            # 强制点击按钮
                            driver.execute_script("arguments[0].click();", retrybutton)
                            print(f"点击'Retry'按钮成功，等待5秒后继续爬取，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                            time.sleep(15)  # 等待15秒后继续爬取
                            attempts = 0
                            previous_post_count = current_post_count
                            break  # 成功点击后跳出循环

                        except Exception as e:
                            # print(f"未能找到'Retry'按钮或点击失败，错误信息: {e}")
                            print("没找到按钮")

                print("多次滚动后没有新数据，等30秒")
                time.sleep(30)

            if attempts > max_attempts:  # 如果连续6次没有新数据，停止爬取
                print(f"多次滚动后没有新数据，已尝试 {attempts}")
                # break
                user_input = input("请输入0停止爬取，1继续爬取: ")  # 获取用户输入
                if user_input == "0":
                    print("停止爬取。")
                    break  # 停止爬取
                elif user_input == "1":
                    print("继续爬取...")
                    attempts = 0  # 重置尝试次数，继续爬取
                else:
                    print("无效输入，请输入0或1。")

            driver.execute_script(f"window.scrollBy(0, {scroll_step});")
            scroll_count += 1
            time.sleep(random.uniform(2, 3))  # 等待加载更多内容




In [14]:
get_posts_byday('momo1', '2024-01-01', '2025-03-21', False)

File C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_momo1_2024-01-01_2025-03-21.json created with initial data.
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_momo1_2024-01-01_2025-03-21.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_momo1_2024-01-01_2025-03-21.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_momo1_2024-01-01_2025-03-21.json
已收集到 4 个帖子
已收集到 4 个帖子
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_momo1_2024-01-01_2025-03-21.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_momo1_2024-01-01_2025-03-21.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_momo1_2024-01-01_2025-03-21.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_momo1_2024-01-01_2025-03-21.json
Data append

KeyboardInterrupt: 

In [4]:
from datetime import datetime, timedelta

def get_posts_day_by_day(keyword, start_date, end_date, live_flag, search_by_year=False):
    # 转换日期字符串为 datetime 对象
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    delta = 15
    if search_by_year:
        delta = 365

    # 循环遍历每一天
    current_date = start_date
    if live_flag:
        print("正在爬取latest搜索结果...")
        while current_date < end_date:
            # 格式化日期为字符串（你可以根据需要调整格式）
            start_date_str = current_date.strftime("%Y-%m-%d")
            end_date_str = (current_date + timedelta(days=delta)).strftime("%Y-%m-%d")
            print(f"爬取日期：{start_date_str}~{end_date_str}")

            # 调用爬虫函数获取当天的数据
            get_posts_byday(keyword, start_date_str, end_date_str, live_flag)

            # 增加一天
            current_date += timedelta(days=delta)
    else:
        print("正在爬取top搜索结果...")
        while current_date < end_date:
            # 格式化日期为字符串（你可以根据需要调整格式）
            start_date_str = current_date.strftime("%Y-%m-%d")
            end_date_str = (current_date + timedelta(days=delta)).strftime("%Y-%m-%d")
            print(f"爬取日期：{start_date_str}~{end_date_str}")

            # 调用爬虫函数获取当天的数据
            get_posts_byday(keyword, start_date_str, end_date_str, live_flag)

            # 增加一天
            current_date += timedelta(days=delta)

# 调用函数
# 采集历史记录
# get_posts_day_by_day('trump garbage', '2024-10-25', '2024-11-07', False)
# get_posts_day_by_day('biden garbage', '2024-10-25', '2024-11-07', False)

In [ ]:
get_posts_day_by_day('pepe the frog', '2020-09-01', '2020-12-31', False)

正在爬取top搜索结果...
爬取日期：2020-09-01~2020-09-16
已收集到 0 个帖子
File C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_pepe the frog_2020-09-01_2020-09-16.json created with initial data.
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_pepe the frog_2020-09-01_2020-09-16.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_pepe the frog_2020-09-01_2020-09-16.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_pepe the frog_2020-09-01_2020-09-16.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_pepe the frog_2020-09-01_2020-09-16.json
已收集到 5 个帖子
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_pepe the frog_2020-09-01_2020-09-16.json
Data appended to C:\Users\admin\codespace\twittercrawler\twitter\json\[top rank]posts_pepe the frog_2020-09-01_2020-09-16.json
Data appended to C:\Users\admin\

In [17]:
# get_posts_day_by_day('biden', '2018-10-25', '2019-11-07', False, False)

In [18]:
# get_posts_day_by_day('genetically modified organism', '2014-01-01', '2023-01-01', False, True)

正在爬取top搜索结果...
爬取日期：2014-01-01~2015-01-01
0
0
48
158
210
219
803
1194
1618
2138
2761
3187
3830
4244
4363
4482
4906
5549
已收集到 12 个帖子
219
803
1194
1618
2138
2761
3187
3830
4244
4363
4482
4906
5549
5942
6499
7142
7654
8078
8721
9364
9483
9604
10081
已收集到 22 个帖子
4482
4906
5549
5942
6499
7142
7654
8078
8721
9364
9483
9604
10081
10593
10984
11519
11999
12118
12787
13267
13386
13505
13624
13743
13862
14382
14501
已收集到 36 个帖子
8721
9364
9483
9604
10081
10593
10984
11519
11999
12118
12787
13267
13386
13505
13624
13743
13862
14382
14501
15030
15550
15671
16095
16214
16333
16812
16931
17050
17169
17288
17781
18159
18278
18397
18516
18635
18734
19246
已收集到 55 个帖子
13743
13862
14382
14501
15030
15550
15671
16095
16214
16333
16812
16931
17050
17169
17288
17781
18159
18278
18397
18516
18635
18734
19246
19758
19897
20418
20537
20656
20815
20934
21053
21457
21576
21735
22159
22623
22742
22861
22980
23404
已收集到 72 个帖子
18278
18397
18516
18635
18734
19246
19758
19897
20418
20537
20656
20815
20934
21053
21457
21

In [19]:
# get_posts_day_by_day('childless cat', '2024-09-11', '2024-09-17', False)

In [20]:
# get_posts_day_by_day('childless cat', '2024-09-11', '2024-09-17', True)

In [21]:
# get_posts_day_by_day('childless cat', '2024-09-15', '2024-09-17', True)

In [90]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import random
import json

#先运行命令再运行此代码块 & "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9530 --user-data-dir="D:\selenium\AutomationProfile"
chromedriver_path = "C:/Program Files/Google/Chrome/Application/chromedriver.exe"

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9530")

# 创建 ChromeDriver 服务
service = Service(chromedriver_path)

# 初始化 WebDriver
driver = webdriver.Chrome(service=service, options=option)

# 去除 WebDriver 痕迹
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    });
    """
})

# 设置隐式等待
driver.implicitly_wait(10)

# 测试访问页面
driver.get("https://www.x.com")
print("标题:", driver.title)

标题: X


In [ ]:
from datetime import datetime
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
import os
import re
from datetime import datetime
from bs4 import BeautifulSoup
import time
def get_post_comments(main_user_id, main_post_id):
    posts_lists = []
    post_index = set() #帖子独立标识集合去重
    # 打开目标帖子的URL
    post_url = f'https://x.com/{main_user_id}/status/{main_post_id}'
    driver.get(post_url)
    time.sleep(10)  # 等待页面加载

    comments = []  # 存储评论的列表

    # 滚动页面以加载更多评论（如果需要）
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = 2000  # 每次向下滚动的像素数
    scroll_count = 0
    previous_post_count = 0  # 用于记录上一次的帖子ID数量
    max_attempts = 3  # 当数据条数不变的最大允许次数
    attempts = 0
    garbages = []
    isgarbage = 0
    parent_comment_id = None
    # 保存路径
    current_date = datetime.now().strftime("%Y-%m-%d-%H-%M")
    base_dir = f"twitter_comments_new"
    os.makedirs(base_dir, exist_ok=True)

    posts_file = os.path.join(base_dir, f"[top rank]comments_genetically modified organism_{main_post_id}.json") # 为了我的转基因case特制的文件名
    
    while True:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        post_tags = soup.select('div[data-testid="cellInnerDiv"]')
        for post_tag in post_tags:
            transform_value = post_tag['style'].split('translateY(')[1].split('px)')[0]
            if float(transform_value)>0 and transform_value not in post_index: # todo：做一下真的采集到手后再去重的逻辑
                # post_index.add(transform_value)
                # print(transform_value)

                # 提取帖子发布时间
                time_tag = post_tag.find('time', attrs={'datetime': True})
                if time_tag and 'datetime' in time_tag.attrs:
                    datetime_str = time_tag['datetime']
                    # 解析ISO格式的日期时间字符串
                    dt = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ')
                    # 格式化为“年-月-日 时:分:秒”格式
                    post_time = dt.strftime('%Y-%m-%d %H:%M:%S')
                else:
                    post_time = ''
                if not post_time and post_tag.find('span', class_='css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3'):
                    info_tag = post_tag.find('span', class_='css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3')
                    info = info_tag.select_one('span').text.strip() if info_tag.select_one('span') else ''
                    if info == 'This Post is unavailable.':
                        parent_comment_id = None
                        continue
                
                
                # 提取用户名
                username_tag = post_tag.select_one('div[data-testid="User-Name"]')
                if username_tag:
                    username = username_tag.get_text(strip=True)
                    # 检查用户名是否包含垃圾词
                    if any(garbage in username for garbage in garbages):
                        isgarbage = 1
                        continue  # 跳过当前帖子，不再提取ID
                    username = username_tag.select_one('span').text.strip() if username_tag.select_one('span') else ''
                    # print(username)
                else:
                    # print("no username")
                    parent_comment_id = None
                    continue
                # # 提取用户名
                # username_span = post_tag.find('span', class_='css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3')
                # if username_span:
                #     username = username_span.get_text()
                # else:
                #     username = 'Unknown'
                # print(username)
                isgarbage = 0
                

                # 提取帖子正文
                # post_content = post_tag.select_one('div[data-testid="tweetText"]').text.strip() if post_tag.select_one('div[data-testid="tweetText"]') else ''
            # 提取帖子正文，并保持图片和文字顺序
                post_content = ""
                tweet_text_div = post_tag.select_one('div[data-testid="tweetText"]')
                if tweet_text_div:
                    # 遍历所有子元素，按顺序提取文字、图片和链接
                    for element in tweet_text_div.children:
                        if element.name == 'span':  # 处理文本
                            post_content += element.text.strip() + " "
                        elif element.name == 'img':  # 处理图片
                            alt_text = element.get('alt', '')
                            if alt_text:
                                post_content += f"{alt_text} "
                        elif element.name == 'a':  # 处理链接
                            href = element.get('href', '')
                            if href:
                                post_content += f"{href} "
                                post_content += element.text.strip() + " "
                # print(post_content)

                # 查找包含aria-label的div标签，提取转评赞以及帖子id
                cnt_tag = None
                if post_time < '2023-01-01':
                    # print(post_time)
                    aria_label = post_tag.find('div', attrs={'aria-label': re.compile(r'.*Likes.*')}) # likes views
                    cnt_tag = post_tag.find_all('button', class_='css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l')
                else:
                    aria_label = post_tag.find('div', attrs={'aria-label': re.compile(r'.*views.*')}) # likes views

                if cnt_tag and post_time < '2023-01-01':
                    # print("has cnt_tag")
                    # 从aria-label中提取数据
                    label_text = cnt_tag[3]['aria-label']

                    # 初始化数据
                    replies = reposts = likes = bookmarks = views = 0

                    # 使用正则表达式逐项匹配
                    # if 'replie' in label_text:
                    #     replies = int(re.search(r'(\d+)\s+replie', label_text).group(1))
                    # if 'repost' in label_text:
                    #     reposts = int(re.search(r'(\d+)\s+repost', label_text).group(1))
                    # if 'like' in label_text:
                    #     likes = int(re.search(r'(\d+)\s+like', label_text).group(1))
                    # if 'bookmark' in label_text:
                    #     bookmarks = int(re.search(r'(\d+)\s+bookmark', label_text).group(1))
                    # 修改代码，处理None的情况
                    for t in cnt_tag:
                        if 'Like' in t['aria-label']:
                            label_text = t['aria-label']
                    if 'Like' in label_text:
                        match = re.search(r'(\d+)\s+Like', label_text)
                        likes = int(match.group(1)) if match else 0
                    # print(likes)
                    for t in cnt_tag:
                        if 'Repl' in t['aria-label']:
                            label_text = t['aria-label']
                    if 'Repl' in label_text:
                        match = re.search(r'(\d+)\s+Repl', label_text)
                        replies = int(match.group(1)) if match else 0
                    for t in cnt_tag:
                        if 'repost' in t['aria-label']:
                            label_text = t['aria-label']
                    if 'repost' in label_text:
                        match = re.search(r'(\d+)\s+repost', label_text)
                        reposts = int(match.group(1)) if match else 0
                    views = 0

                    # 查找帖子ID和用户ID
                    post_id_tag = post_tag.find_all('div', class_='css-175oi2r r-18u37iz r-1h0z5md r-13awgt0')
                    if post_id_tag:
                        # 找到第四个div，提取其中的链接
                        post_id_link = post_id_tag[3].find('a', href=True)
                        if post_id_link:
                            post_id = post_id_link['href'].split('/')[3]  # 从URL中提取帖子ID
                            user_id = post_id_link['href'].split('/')[1]
                        else:
                            post_id = ''
                            user_id = ''
                    else:
                        post_id = ''
                        user_id = ''
                    # print(post_id)

                elif aria_label and post_time >= '2023-01-01':
                    # print("has aria_label")
                    # 从aria-label中提取数据
                    label_text = aria_label['aria-label']

                    # 初始化数据
                    replies = reposts = likes = bookmarks = views = 0

                    # 使用正则表达式逐项匹配
                    # if 'replie' in label_text:
                    #     replies = int(re.search(r'(\d+)\s+replie', label_text).group(1))
                    # if 'repost' in label_text:
                    #     reposts = int(re.search(r'(\d+)\s+repost', label_text).group(1))
                    # if 'like' in label_text:
                    #     likes = int(re.search(r'(\d+)\s+like', label_text).group(1))
                    # if 'bookmark' in label_text:
                    #     bookmarks = int(re.search(r'(\d+)\s+bookmark', label_text).group(1))
                    # 修改代码，处理None的情况
                    if 'repl' in label_text:
                        match = re.search(r'(\d+)\s+repl', label_text)
                        replies = int(match.group(1)) if match else 0

                    if 'repost' in label_text:
                        match = re.search(r'(\d+)\s+repost', label_text)
                        reposts = int(match.group(1)) if match else 0

                    if 'like' in label_text:
                        match = re.search(r'(\d+)\s+like', label_text)
                        likes = int(match.group(1)) if match else 0

                    if 'bookmark' in label_text:
                        match = re.search(r'(\d+)\s+bookmark', label_text)
                        bookmarks = int(match.group(1)) if match else 0
                    if 'view' in label_text:
                        # views = int(re.search(r'(\d+)\s+view', label_text).group(1))
                        # 检查是否匹配到了 "view" 和数字
                        match = re.search(r'(\d+)\s+view', label_text)
                        if match:
                            views = int(match.group(1))
                        else:
                            views = 0  # 如果没有匹配到，设置默认值为0

                    # 查找帖子ID和用户ID
                    post_id_tag = aria_label.find_all('div', class_='css-175oi2r r-18u37iz r-1h0z5md r-13awgt0')
                    if post_id_tag:
                        # 找到第四个div，提取其中的链接
                        post_id_link = post_id_tag[3].find('a', href=True)
                        if post_id_link:
                            post_id = post_id_link['href'].split('/')[3]  # 从URL中提取帖子ID
                            user_id = post_id_link['href'].split('/')[1]
                        else:
                            post_id = ''
                            user_id = ''
                    else:
                        post_id = ''
                        user_id = ''
                else:
                    replies = reposts = likes = bookmarks = views = 0
                    post_id = ''
                    user_id = ''
                    parent_comment_id = None
                    break

                # 提取hashtags，确保唯一性，使用集合去重
                hashtags = set()
                hashtag_tags = post_tag.select('a[href*="/hashtag/"]')
                for tag in hashtag_tags:
                    hashtag = tag.get_text(strip=True)
                    hashtags.add(hashtag)

                # 提取图片和视频的链接
                media_urls = set()  # 使用集合来自动去重
                img_tags = post_tag.find_all('img')
                for img in img_tags:
                    img_url = img.get('src')
                    if img_url and 'profile_images' not in img_url and 'emoji' not in img_url:
                        media_urls.add(img_url)
                media_urls = list(media_urls)

                # 提取@了哪些用户
                at_usernames = set()  # 使用集合确保唯一性
                at_usernames.update(re.findall(r'@(\w+)', post_content))  # 查找所有 @ 后的用户名

                # 提取帖子中的所有链接（使用正则表达式从文本中匹配URL）
                urls = set()  # 使用集合去重

                # 匹配以 http 或 https 开头的 URL
                url_pattern = re.compile(r'https?://[^\s]+')

                # 在帖子文本中查找所有符合条件的URL
                urls.update(url_pattern.findall(post_content))

                # 查找回复的多个用户
                reply_users = []
                reply_tag = post_tag.find('div', class_='css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41') # 查找包含回复信息的 div
                if reply_tag:
                    # 获取所有回复用户的 a 标签
                    reply_user_tags = reply_tag.find_all('a')
                    for reply_user_tag in reply_user_tags:
                        # 提取用户名并添加到列表
                        reply_users.append(reply_user_tag.text.strip())

                # 将评论及用户信息存储在字典中
                comment_data = {
                        "postUrl": f"https://x.com/{user_id}/status/{post_id}",
                        "mid": post_id,
                        "text": post_content,
                        "date": post_time,
                        "userId": user_id,
                        "userName": username,
                        "source_post_id": main_post_id,  # 主帖ID
                        "parent_comment_id": parent_comment_id,  # 父级ID
                        "likeNum": likes,
                        "commentNum": replies,
                        "repostNum": reposts,
                        "viewNum": views,
                        "hashtags": list(hashtags),
                        "mediaUrls": media_urls,
                        "atUsernames": list(at_usernames),  # 新增字段：记录@的用户
                        "urls": list(urls),  # 新增字段：记录帖中的所有链接
                        "replyUsers": reply_users  # 新增字段：记录多个回复的用户
                }
                posts_lists.append(comment_data)
                post_index.add(transform_value)

                # 保存 posts
                with open(posts_file, "w", encoding="utf-8") as f:
                    json.dump(list(posts_lists), f, ensure_ascii=False, indent=4)

                reply_line = post_tag.find('div', class_='css-175oi2r r-1bnu78o r-f8sm7e r-m5arl1 r-16y2uox r-14gqq1x')
                if reply_line:
                    parent_comment_id = post_id
                else:
                    parent_comment_id = None

        current_post_count = len(posts_lists)
        print(f"已收集到 {current_post_count} 个评论帖")

        if current_post_count >= 3000:  # 如果爬取到的评论条数超过3000，则停止爬取
             break

        if current_post_count == previous_post_count and not isgarbage: #防止遇到批量垃圾帖子，导致误识别为滑到底了
            time.sleep(random.uniform(1, 2))
            attempts += 1
        else:
            attempts = 0
            previous_post_count = current_post_count

        if attempts == max_attempts:
            # 检测到页面错误信息，尝试点击'Retry'按钮...
            if len(post_tags) > 0 and post_tags[-1].select_one('span').text.strip() == "Something went wrong. Try reloading." if post_tags[-1].select_one('span') else '':
                print(f"检测到'Retry'按钮，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')},滚动了{scroll_count}次")
                # 等待人工确认点击
                # input("按Enter键以继续爬取...")
                while True:  # 无限循环，直到成功点击
                    try:
                        # 等待6分钟（360秒）
                        print("等待6分钟...")
                        time.sleep(360)  # 等待6分钟

                        # 使用 CSS Selector 或 XPath 查找'Retry'按钮并确保它可点击
                        retrybutton = WebDriverWait(driver, 3).until(
                            EC.element_to_be_clickable((By.XPATH, "//button[.//*[contains(text(),'Retry')]]"))
                        )

                        # 滚动到按钮所在位置
                        driver.execute_script("arguments[0].scrollIntoView();", retrybutton)

                        # 强制点击按钮
                        driver.execute_script("arguments[0].click();", retrybutton)
                        print(f"点击'Retry'按钮成功，等待5秒后继续爬取，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                        time.sleep(15)  # 等待15秒后继续爬取
                        attempts = 0
                        previous_post_count = current_post_count
                        break  # 成功点击后跳出循环

                    except Exception as e:
                        # print(f"未能找到'Retry'按钮或点击失败，错误信息: {e}")
                        print("没找到按钮")

            print("多次滚动后没有新数据，等10秒")
            time.sleep(10)

        if attempts > max_attempts:  # 如果连续6次没有新数据，停止爬取
            print(f"多次滚动后没有新数据，已尝试 {attempts}")
            break
            # user_input = input("请输入0停止爬取，1继续爬取: ")  # 获取用户输入
            # if user_input == "0":
            #     print("停止爬取。")
            #     break  # 停止爬取
            # elif user_input == "1":
            #     print("继续爬取...")
            #     attempts = 0  # 重置尝试次数，继续爬取
            # else:
            #     print("无效输入，请输入0或1。")

        driver.execute_script(f"window.scrollBy(0, {scroll_step});")
        scroll_count += 1
        time.sleep(random.uniform(2, 3))  # 等待加载更多内容

# 测试爬取一个帖子的评论
# get_post_comments('kohkalah', '1804259338766241930')
# get_post_comments('RFKJrHealthSec', '1867589034274340942')

In [100]:
import os

folder_path = 'twitter_posts_new'  # 替换为你的文件夹路径
posturls_file = '[top rank]posturls_genetically modified organism_2014_2025.txt'

# 用于存储读取的postUrls
post_urls = []

# 读取文件中的每一行，去掉换行符并存储到列表中
with open(os.path.join(folder_path, posturls_file), 'r', encoding='utf-8') as file:
    post_urls = [line.strip() for line in file.readlines()]
print(len(post_urls))

966


In [10]:
test_url = 'https://x.com/UNAM_MX/status/995121432135905280'
print(test_url.split('/')[3])
print(test_url.split('/')[5])

UNAM_MX
995121432135905280


In [99]:
#从ptr_url.txt中读取ptr_url
with open(os.path.join(folder_path,'ptr_url.txt'), 'r') as file:
    ptr_url = [line.strip() for line in file.readlines()][0]
print(ptr_url)

https://x.com/Nature/status/718467367106383872


In [ ]:
timesleep_cnt = 0
after_ptr_url = False
for post_url in post_urls:
    if post_url == ptr_url:
        print("到达指针位置")
        after_ptr_url = True
    if after_ptr_url:
        user_id = post_url.split('/')[3]
        post_id = post_url.split('/')[5]
        print(f"正在爬取帖子评论：{user_id}/{post_id}")
        ptr_url = post_url
        # 存储ptr_url到文件
        with open(os.path.join(folder_path, 'ptr_url.txt'), 'w', encoding='utf-8') as file:
            file.write(ptr_url)
        get_post_comments(user_id, post_id)
        timesleep_cnt += 1
        if timesleep_cnt % 20 == 0:
            print(f"已爬取{timesleep_cnt}个主帖的评论，休息5分钟...")
            time.sleep(300)
        else:
            time.sleep(5)
    else:
        print(f"{post_url}已被采集过评论")

In [103]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import random
import json

#先运行命令再运行此代码块 & "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9530 --user-data-dir="D:\selenium\AutomationProfile"
chromedriver_path = "C:/Program Files/Google/Chrome/Application/chromedriver.exe"

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9530")

# 创建 ChromeDriver 服务
service = Service(chromedriver_path)

# 初始化 WebDriver
driver = webdriver.Chrome(service=service, options=option)

# 去除 WebDriver 痕迹
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    });
    """
})

# 设置隐式等待
driver.implicitly_wait(10)

# 测试访问页面
driver.get("https://www.x.com")
print("标题:", driver.title)

标题: X


In [146]:
from datetime import datetime
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver import ActionChains
import os
import re
from datetime import datetime
from bs4 import BeautifulSoup
import time
def get_post_quotes(main_user_id, main_post_id):
    posts_lists = []
    post_index = set() #帖子独立标识集合去重
    # 打开目标帖子的URL
    post_url = f'https://x.com/{main_user_id}/status/{main_post_id}/quotes'
    driver.get(post_url)
    time.sleep(10)  # 等待页面加载

    quotes = []  # 存储评论的列表

    # 滚动页面以加载更多评论（如果需要）
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = 4500  # 每次向下滚动的像素数
    scroll_count = 0
    previous_post_count = 0  # 用于记录上一次的帖子ID数量
    max_attempts = 3  # 当数据条数不变的最大允许次数
    attempts = 0
    garbages = []
    isgarbage = 0
    # 保存路径
    current_date = datetime.now().strftime("%Y-%m-%d-%H-%M")
    base_dir = f"twitter_quotes"
    os.makedirs(base_dir, exist_ok=True)

    posts_file = os.path.join(base_dir, f"[top rank]quotes_genetically modified organism_{main_post_id}.json") # 为了我的转基因case特制的文件名
    
    while True:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        empty_tag = soup.select('div[data-testid="empty_state_header_text"]')
        if empty_tag:
            print("no quotes")
            break
        post_tags = soup.select('div[data-testid="cellInnerDiv"]')
        for post_tag in post_tags:
            transform_value = post_tag['style'].split('translateY(')[1].split('px)')[0]
            if transform_value not in post_index: # todo：做一下真的采集到手后再去重的逻辑
                # post_index.add(transform_value)
                # print(transform_value)

                info_tag = post_tag.find('div', class_='css-175oi2r r-k4xj1c r-18u37iz r-1wtj0ep')
                if not info_tag:
                    continue
                # 提取帖子发布时间
                time_tag = info_tag.find('time', attrs={'datetime': True})
                if time_tag and 'datetime' in time_tag.attrs:
                    datetime_str = time_tag['datetime']
                    # 解析ISO格式的日期时间字符串
                    dt = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ')
                    # 格式化为“年-月-日 时:分:秒”格式
                    post_time = dt.strftime('%Y-%m-%d %H:%M:%S')
                else:
                    post_time = ''
                
                
                # 提取用户名
                username_tag = info_tag.select_one('div[data-testid="User-Name"]')
                if username_tag:
                    username = username_tag.get_text(strip=True)
                    # 检查用户名是否包含垃圾词
                    if any(garbage in username for garbage in garbages):
                        isgarbage = 1
                        continue  # 跳过当前帖子，不再提取ID
                    username = username_tag.select_one('span').text.strip() if username_tag.select_one('span') else ''
                    # print(username)
                else:
                    # print("no username")
                    continue
                # # 提取用户名
                # username_span = post_tag.find('span', class_='css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3')
                # if username_span:
                #     username = username_span.get_text()
                # else:
                #     username = 'Unknown'
                # print(username)
                isgarbage = 0
                

                # 提取帖子正文
                # post_content = post_tag.select_one('div[data-testid="tweetText"]').text.strip() if post_tag.select_one('div[data-testid="tweetText"]') else ''
            # 提取帖子正文，并保持图片和文字顺序
                post_content = ""
                quote_tag = post_tag.find('div', style="-webkit-line-clamp: 10; color: rgb(231, 233, 234);")
                if not quote_tag:
                    continue
                if quote_tag:
                    # 遍历所有子元素，按顺序提取文字、图片和链接
                    for element in quote_tag.children:
                        if element.name == 'span':  # 处理文本
                            post_content += element.text.strip() + " "
                        elif element.name == 'img':  # 处理图片
                            alt_text = element.get('alt', '')
                            if alt_text:
                                post_content += f"{alt_text} "
                        elif element.name == 'a':  # 处理链接
                            href = element.get('href', '')
                            if href:
                                post_content += f"{href} "
                                post_content += element.text.strip() + " "
                # print(post_content)

                # 查找包含aria-label的div标签，提取转评赞以及帖子id
                cnt_tag = None
                if post_time < '2023-01-01':
                    # print(post_time)
                    aria_label = post_tag.find('div', attrs={'aria-label': re.compile(r'.*Likes.*')}) # likes views
                    cnt_tag = post_tag.find_all('button', class_='css-175oi2r r-1777fci r-bt1l66 r-bztko3 r-lrvibr r-1loqt21 r-1ny4l3l')
                else:
                    aria_label = post_tag.find('div', attrs={'aria-label': re.compile(r'.*views.*')}) # likes views

                if cnt_tag and post_time < '2023-01-01':
                    # print("has cnt_tag")
                    # 从aria-label中提取数据
                    label_text = cnt_tag[3]['aria-label']

                    # 初始化数据
                    replies = reposts = likes = bookmarks = views = 0

                    # 使用正则表达式逐项匹配
                    # if 'replie' in label_text:
                    #     replies = int(re.search(r'(\d+)\s+replie', label_text).group(1))
                    # if 'repost' in label_text:
                    #     reposts = int(re.search(r'(\d+)\s+repost', label_text).group(1))
                    # if 'like' in label_text:
                    #     likes = int(re.search(r'(\d+)\s+like', label_text).group(1))
                    # if 'bookmark' in label_text:
                    #     bookmarks = int(re.search(r'(\d+)\s+bookmark', label_text).group(1))
                    # 修改代码，处理None的情况
                    for t in cnt_tag:
                        if 'Like' in t['aria-label']:
                            label_text = t['aria-label']
                    if 'Like' in label_text:
                        match = re.search(r'(\d+)\s+Like', label_text)
                        likes = int(match.group(1)) if match else 0
                    # print(likes)
                    for t in cnt_tag:
                        if 'Repl' in t['aria-label']:
                            label_text = t['aria-label']
                    if 'Repl' in label_text:
                        match = re.search(r'(\d+)\s+Repl', label_text)
                        replies = int(match.group(1)) if match else 0
                    for t in cnt_tag:
                        if 'repost' in t['aria-label']:
                            label_text = t['aria-label']
                    if 'repost' in label_text:
                        match = re.search(r'(\d+)\s+repost', label_text)
                        reposts = int(match.group(1)) if match else 0
                    views = 0

                    # 查找帖子ID和用户ID
                    post_id_tag = post_tag.find_all('div', class_='css-175oi2r r-18u37iz r-1h0z5md r-13awgt0')
                    if post_id_tag:
                        # 找到第四个div，提取其中的链接
                        post_id_link = post_id_tag[3].find('a', href=True)
                        if post_id_link:
                            post_id = post_id_link['href'].split('/')[3]  # 从URL中提取帖子ID
                            user_id = post_id_link['href'].split('/')[1]
                        else:
                            post_id = ''
                            user_id = ''
                    else:
                        post_id = ''
                        user_id = ''
                    # print(post_id)

                elif aria_label and post_time >= '2023-01-01':
                    # print("has aria_label")
                    # 从aria-label中提取数据
                    label_text = aria_label['aria-label']

                    # 初始化数据
                    replies = reposts = likes = bookmarks = views = 0

                    # 使用正则表达式逐项匹配
                    # if 'replie' in label_text:
                    #     replies = int(re.search(r'(\d+)\s+replie', label_text).group(1))
                    # if 'repost' in label_text:
                    #     reposts = int(re.search(r'(\d+)\s+repost', label_text).group(1))
                    # if 'like' in label_text:
                    #     likes = int(re.search(r'(\d+)\s+like', label_text).group(1))
                    # if 'bookmark' in label_text:
                    #     bookmarks = int(re.search(r'(\d+)\s+bookmark', label_text).group(1))
                    # 修改代码，处理None的情况
                    if 'repl' in label_text:
                        match = re.search(r'(\d+)\s+repl', label_text)
                        replies = int(match.group(1)) if match else 0

                    if 'repost' in label_text:
                        match = re.search(r'(\d+)\s+repost', label_text)
                        reposts = int(match.group(1)) if match else 0

                    if 'like' in label_text:
                        match = re.search(r'(\d+)\s+like', label_text)
                        likes = int(match.group(1)) if match else 0

                    if 'bookmark' in label_text:
                        match = re.search(r'(\d+)\s+bookmark', label_text)
                        bookmarks = int(match.group(1)) if match else 0
                    if 'view' in label_text:
                        # views = int(re.search(r'(\d+)\s+view', label_text).group(1))
                        # 检查是否匹配到了 "view" 和数字
                        match = re.search(r'(\d+)\s+view', label_text)
                        if match:
                            views = int(match.group(1))
                        else:
                            views = 0  # 如果没有匹配到，设置默认值为0

                    # 查找帖子ID和用户ID
                    post_id_tag = aria_label.find_all('div', class_='css-175oi2r r-18u37iz r-1h0z5md r-13awgt0')
                    if post_id_tag:
                        # 找到第四个div，提取其中的链接
                        post_id_link = post_id_tag[3].find('a', href=True)
                        if post_id_link:
                            post_id = post_id_link['href'].split('/')[3]  # 从URL中提取帖子ID
                            user_id = post_id_link['href'].split('/')[1]
                        else:
                            post_id = ''
                            user_id = ''
                    else:
                        post_id = ''
                        user_id = ''
                else:
                    replies = reposts = likes = bookmarks = views = 0
                    post_id = ''
                    user_id = ''
                    parent_comment_id = None
                    break

                # 提取hashtags，确保唯一性，使用集合去重
                hashtags = set()
                hashtag_tags = quote_tag.select('a[href*="/hashtag/"]')
                for tag in hashtag_tags:
                    hashtag = tag.get_text(strip=True)
                    hashtags.add(hashtag)

                # 提取图片和视频的链接
                media_urls = set()  # 使用集合来自动去重
                img_tags = quote_tag.find_all('img')
                for img in img_tags:
                    img_url = img.get('src')
                    if img_url and 'profile_images' not in img_url and 'emoji' not in img_url:
                        media_urls.add(img_url)
                media_urls = list(media_urls)

                # 提取@了哪些用户
                at_usernames = set()  # 使用集合确保唯一性
                at_usernames.update(re.findall(r'@(\w+)', post_content))  # 查找所有 @ 后的用户名

                # 提取帖子中的所有链接（使用正则表达式从文本中匹配URL）
                urls = set()  # 使用集合去重

                # 匹配以 http 或 https 开头的 URL
                url_pattern = re.compile(r'https?://[^\s]+')

                # 在帖子文本中查找所有符合条件的URL
                urls.update(url_pattern.findall(post_content))

                # 查找回复的多个用户
                reply_users = []
                reply_tag = quote_tag.find('div', class_='css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41') # 查找包含回复信息的 div
                if reply_tag:
                    # 获取所有回复用户的 a 标签
                    reply_user_tags = reply_tag.find_all('a')
                    for reply_user_tag in reply_user_tags:
                        # 提取用户名并添加到列表
                        reply_users.append(reply_user_tag.text.strip())

                # 将评论及用户信息存储在字典中
                quote_data = {
                        "postUrl": f"https://x.com/{user_id}/status/{post_id}",
                        "mid": post_id,
                        "text": post_content,
                        "date": post_time,
                        "userId": user_id,
                        "userName": username,
                        "source_post_id": main_post_id,  # 主帖ID
                        "is_quote": True,  # 父级ID
                        "likeNum": likes,
                        "commentNum": replies,
                        "repostNum": reposts,
                        "viewNum": views,
                        "hashtags": list(hashtags),
                        "mediaUrls": media_urls,
                        "atUsernames": list(at_usernames),  # 新增字段：记录@的用户
                        "urls": list(urls),  # 新增字段：记录帖中的所有链接
                        "replyUsers": reply_users  # 新增字段：记录多个回复的用户
                }
                posts_lists.append(quote_data)
                post_index.add(transform_value)

                # 保存 posts
                with open(posts_file, "w", encoding="utf-8") as f:
                    json.dump(list(posts_lists), f, ensure_ascii=False, indent=4)

        current_post_count = len(posts_lists)
        print(f"已收集到 {current_post_count} 个quote帖")

        if current_post_count >= 3000:  # 如果爬取到的quote条数超过3000，则停止爬取
             break

        if current_post_count == previous_post_count and not isgarbage: #防止遇到批量垃圾帖子，导致误识别为滑到底了
            time.sleep(random.uniform(1, 2))
            attempts += 1
        else:
            attempts = 0
            previous_post_count = current_post_count

        if attempts == max_attempts:
            # 检测到页面错误信息，尝试点击'Retry'按钮...
            if len(post_tags) == 0:
                print(f"检测到'Retry'按钮，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')},滚动了{scroll_count}次")
                # 等待人工确认点击
                # input("按Enter键以继续爬取...")
                while True:  # 无限循环，直到成功点击
                    try:
                        # 等待6分钟（360秒）
                        print("等待6分钟...")
                        time.sleep(360)  # 等待6分钟

                        # 使用 CSS Selector 或 XPath 查找'Retry'按钮并确保它可点击
                        retrybutton = WebDriverWait(driver, 3).until(
                            EC.element_to_be_clickable((By.XPATH, "//button[.//*[contains(text(),'Retry')]]"))
                        )

                        # 滚动到按钮所在位置
                        driver.execute_script("arguments[0].scrollIntoView();", retrybutton)

                        # 强制点击按钮
                        driver.execute_script("arguments[0].click();", retrybutton)
                        print(f"点击'Retry'按钮成功，等待5秒后继续爬取，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                        time.sleep(15)  # 等待15秒后继续爬取
                        attempts = 0
                        previous_post_count = current_post_count
                        break  # 成功点击后跳出循环

                    except Exception as e:
                        # print(f"未能找到'Retry'按钮或点击失败，错误信息: {e}")
                        print("没找到按钮")

            elif len(post_tags) > 0 and post_tags[-1].select_one('span').text.strip() == "Something went wrong. Try reloading." if post_tags[-1].select_one('span') else '':
                print(f"检测到'Retry'按钮，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')},滚动了{scroll_count}次")
                # 等待人工确认点击
                # input("按Enter键以继续爬取...")
                while True:  # 无限循环，直到成功点击
                    try:
                        # 等待6分钟（360秒）
                        print("等待6分钟...")
                        time.sleep(360)  # 等待6分钟

                        # 使用 CSS Selector 或 XPath 查找'Retry'按钮并确保它可点击
                        retrybutton = WebDriverWait(driver, 3).until(
                            EC.element_to_be_clickable((By.XPATH, "//button[.//*[contains(text(),'Retry')]]"))
                        )

                        # 滚动到按钮所在位置
                        driver.execute_script("arguments[0].scrollIntoView();", retrybutton)

                        # 强制点击按钮
                        driver.execute_script("arguments[0].click();", retrybutton)
                        print(f"点击'Retry'按钮成功，等待5秒后继续爬取，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                        time.sleep(15)  # 等待15秒后继续爬取
                        attempts = 0
                        previous_post_count = current_post_count
                        break  # 成功点击后跳出循环

                    except Exception as e:
                        # print(f"未能找到'Retry'按钮或点击失败，错误信息: {e}")
                        print("没找到按钮")

            print("多次滚动后没有新数据，等10秒")
            time.sleep(10)

        if attempts > max_attempts:  # 如果连续6次没有新数据，停止爬取
            print(f"多次滚动后没有新数据，已尝试 {attempts}")
            break
            # user_input = input("请输入0停止爬取，1继续爬取: ")  # 获取用户输入
            # if user_input == "0":
            #     print("停止爬取。")
            #     break  # 停止爬取
            # elif user_input == "1":
            #     print("继续爬取...")
            #     attempts = 0  # 重置尝试次数，继续爬取
            # else:
            #     print("无效输入，请输入0或1。")

        driver.execute_script(f"window.scrollBy(0, {scroll_step});")
        scroll_count += 1
        time.sleep(random.uniform(2, 3))  # 等待加载更多内容

# 测试爬取一个帖子的评论
# get_post_comments('kohkalah', '1804259338766241930')
# get_post_comments('RFKJrHealthSec', '1867589034274340942')

In [128]:
get_post_quotes("OluchiJudith10", "1892901957351747733")

no quotes


In [ ]:
import os

folder_path = 'twitter_posts_new'  # 替换为你的文件夹路径
posturls_file = '[top rank]posturls_wquotes_genetically modified organism_2014_2025.txt'

# 用于存储读取的postUrls
post_urls = []

# 读取文件中的每一行，去掉换行符并存储到列表中
with open(os.path.join(folder_path, posturls_file), 'r', encoding='utf-8') as file:
    post_urls = [line.strip() for line in file.readlines()]
print(len(post_urls))

In [ ]:
#从ptr_url.txt中读取ptr_url
with open(os.path.join(folder_path,'ptr_quote_url.txt'), 'r') as file:
    file_content = file.readlines()
    ptr_quote_url = file_content[0] if len(file_content) else ''
print(ptr_quote_url)

In [ ]:
timesleep_cnt = 0
after_ptr_url = False
for post_url in post_urls:
    if post_url == ptr_quote_url or ptr_quote_url == '':
        print("到达指针位置")
        after_ptr_url = True
    if after_ptr_url:
        user_id = post_url.split('/')[3]
        post_id = post_url.split('/')[5]
        print(f"正在爬取帖子quote：{user_id}/{post_id}")
        ptr_quote_url = post_url
        # 存储ptr_quote_url到文件
        with open(os.path.join(folder_path, 'ptr_quote_url.txt'), 'w', encoding='utf-8') as file:
            file.write(ptr_quote_url)
        get_post_quotes(user_id, post_id)
        timesleep_cnt += 1
        if timesleep_cnt % 50 == 0:
            print(f"已爬取{timesleep_cnt}个主帖的quote，休息2分钟...")
            time.sleep(120)
        else:
            time.sleep(3)
    else:
        print(f"{post_url}已被采集过quote")

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import random
import time
import json

In [2]:
#先运行命令再运行此代码块 & "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9530 --user-data-dir="D:\selenium\AutomationProfile"
chromedriver_path = "C:/Program Files/Google/Chrome/Application/chromedriver.exe"

option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9530")

# 创建 ChromeDriver 服务
service = Service(chromedriver_path)

# 初始化 WebDriver
driver = webdriver.Chrome(service=service, options=option)

# 去除 WebDriver 痕迹
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    });
    """
})

# 设置隐式等待
driver.implicitly_wait(10)

# 测试访问页面
driver.get("https://www.x.com")
print("标题:", driver.title)


标题: X


In [3]:
#修正数字
import re
def parse_number(value):
    try:
        # 使用正则表达式提取数字和小数部分
        match = re.search(r'[\d.]+', value)
        if match:
            number = match.group()
            if 'K' in value:
                return int(float(number) * 1000)
            elif 'M' in value:
                return int(float(number) * 1000000)
            elif 'B' in value:
                return int(float(number) * 100000000)
            else:
                return int(float(number))  # 支持小数部分
        else:
            raise ValueError("未找到数值部分")
    except (ValueError, TypeError) as e:
        print(f"无法解析数值：{value}，错误：{e}")
        return value  # 返回原始值

In [4]:
#获取个人主页信息
from tqdm import tqdm
import os
import re
from datetime import datetime
from bs4 import BeautifulSoup
import time

#单个用户主页信息采集
def get_user_profiles(profile_url, user_id):
    # 打开指定的个人主页链接
    driver.get(profile_url)
    user_info = {}

    try:
        # 等待导航栏标签加载完成
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//nav[@aria-label='Profile timelines']"))
        )
    except Exception as e:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        #检查是否为异常账户
        if soup.find('div', {'data-testid': 'emptyState'}):
            return 1  #识别为异常账户
        return 0

    time.sleep(random.uniform(3, 4.5))

    # 获取页面 HTML
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # 获取用户名
    username_tag = soup.find('div', {'data-testid': 'UserName'})
    if username_tag:
        user_info['userName'] = username_tag.find('span').text.strip()

    # 获取id
    user_info['userId'] = user_id

    # 获取intro
    nickname_tag = soup.find('div', {'data-testid': 'UserDescription'})
    if nickname_tag:
        user_info['intro'] = nickname_tag.text.strip()
    else:
        user_info['intro'] =  ""

    #获取Profession职业信息
    profession_tag = soup.find('span', {'data-testid': 'UserProfessionalCategory'})
    if profession_tag:
        user_info['profession'] = profession_tag.text.strip()

    #获取birthdate
    birthdate_tag = soup.find('span', {'data-testid': 'UserBirthdate'})
    if birthdate_tag:
        user_info['birthdate'] = birthdate_tag.text.strip()

    # 获取用户网址
    user_url_tag = soup.find('a', {'data-testid': 'UserUrl'})
    if user_url_tag:
        user_info['userUrl'] = user_url_tag['href'].strip()

    # 获取关注数
    following_tag = soup.find('a', {'href': lambda x: x and '/following' in x})
    if following_tag:
        follow_count_text = following_tag.find('span', recursive=False).text.strip() if following_tag.find('span',
                                                                                                           recursive=False) else following_tag.text.strip()
        user_info['following'] = parse_number(follow_count_text)

    # 获取粉丝数
    followers_tag = soup.find('a', {'href': lambda x: x and '/verified_followers' in x})
    if followers_tag:
        followers_count_text = followers_tag.find('span', recursive=False).text.strip() if followers_tag.find('span',
                                                                                                              recursive=False) else followers_tag.text.strip()
        user_info['followers'] = parse_number(followers_count_text)

    # 获取订阅数
    subscriptions_tag = soup.find('a', {'href': lambda x: x and '/subscriptions' in x})
    if subscriptions_tag:
        subscriptions_count_text = subscriptions_tag.find('span', recursive=False).text.strip() if subscriptions_tag.find(
            'span', recursive=False) else subscriptions_tag.text.strip()
        user_info['subscriptions'] = parse_number(subscriptions_count_text)

    # 获取地点
    location_tag = soup.find('span', {'data-testid': 'UserLocation'})
    if location_tag:
        user_info['location'] = location_tag.text.strip()

    return user_info

#遍历采集，等待机制
def start_get_user_profiles(user_ids):

    print(f"需要爬取信息的用户ID总数: {len(user_ids)}个")

    # 创建存储用户资料信息的目录
    base_dir = f"./GMO/users"
    os.makedirs(base_dir, exist_ok=True)

    current_date = datetime.now().strftime("%Y-%m-%d-%H-%M") 
    # 确定已有文件的初始状态
    current_file_path = os.path.join(base_dir, f"user_profiles_{current_date}.json")
    user_profiles = []

    if not os.path.exists(current_file_path):
        with open(current_file_path, 'w', encoding='utf-8') as f:
            json.dump([], f)  # 写入空数组
    else:
        with open(current_file_path, 'r', encoding='utf-8') as f:
            user_profiles = json.load(f)

    current_index = len(user_profiles)

    # 记录上次保存的数量
    last_saved_count = len(user_profiles)

    # 从当前索引位置开始爬取用户信息
    user_ids = user_ids[current_index:]
    print(f"开始爬取用户主页信息，当前已有数据量: {current_index}")

    # 获取用户信息
    with tqdm(total=len(user_ids), desc="获取用户信息进度", unit="用户") as pbar:
        for idx, user_id in enumerate(user_ids):
            profile_url = f'https://x.com/{user_id}'
            retrys = 6
            for retry in range(retrys):
                user_info = get_user_profiles(profile_url, user_id)
                if user_info:
                    if user_info == 1:  # 识别出异常账户，跳过
                        break
                    user_profiles.append(user_info)
                    break
                else:
                    if retry == 0:
                        print(f"第{retry + 1}次尝试，未能获取到用户 {user_id} 的信息, 休息20秒")
                        time.sleep(20)
                    elif retry == 5:
                        print(f"获取 {user_id} 的信息失败，跳过")
                    else:
                        print(f"第{retry + 1}次尝试，未能获取到用户 {user_id} 的信息, 休息六分钟，当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                        time.sleep(365)

            # 每次到达 20 个用户信息时，保存一次
            if len(user_profiles) >= last_saved_count + 20:
                with open(current_file_path, "w", encoding="utf-8") as f:
                    json.dump(user_profiles, f, ensure_ascii=False, indent=4)
                last_saved_count = len(user_profiles)
                print(f"已保存 {len(user_profiles)} 个用户主页信息到 {current_file_path}")

            time.sleep(random.uniform(1, 1.5))  # 随机休息
            pbar.update(1)

    # 保存剩余的用户信息（不足 20 个的部分）
    if len(user_profiles) > last_saved_count:
        with open(current_file_path, "w", encoding="utf-8") as f:
            json.dump(user_profiles, f, ensure_ascii=False, indent=4)
        print(f"已保存剩余的 {len(user_profiles) - last_saved_count} 个用户主页信息到 {current_file_path}")

    print("所有用户主页信息已保存完毕")


In [5]:
# 将GMO/user_id.txt中的内容load到user_ids
user_ids = []
with open('GMO/[top rank]userids_genetically modified organism_2014_2025.txt', 'r', encoding='utf-8') as f:
    user_ids = [line.strip() for line in f.readlines()]
print(len(user_ids))

25233


In [6]:
# 把ptr_user.txt中的内容加载到ptr_user中
with open('GMO/users/ptr_user.txt', 'r') as file:
    file_content = file.readlines()
    ptr_user = file_content[0] if len(file_content) else ''
i = 0
for user_id in user_ids:
    if user_id == ptr_user:
        break
    i = i + 1
user_ids = user_ids[i:]
print(len(user_ids))

25037


In [ ]:
start_get_user_profiles(user_ids)

需要爬取信息的用户ID总数: 25233个
开始爬取用户主页信息，当前已有数据量: 0


获取用户信息进度:   0%|          | 20/25233 [02:16<42:42:35,  6.10s/用户]

已保存 20 个用户主页信息到 ./GMO/users\user_profiles.json


获取用户信息进度:   0%|          | 40/25233 [04:21<48:16:14,  6.90s/用户]

已保存 40 个用户主页信息到 ./GMO/users\user_profiles.json


...

已保存 160 个用户主页信息到 ./GMO/users\user_profiles.json


获取用户信息进度:   1%|          | 180/25233 [20:28<53:34:40,  7.70s/用户]

已保存 180 个用户主页信息到 ./GMO/users\user_profiles.json


获取用户信息进度:   1%|          | 196/25233 [22:15<45:30:50,  6.54s/用户]

第1次尝试，未能获取到用户 2w1t2w1tter 的信息, 休息20秒
第2次尝试，未能获取到用户 2w1t2w1tter 的信息, 休息六分钟，当前时间: 2025-02-25 14:14:53
